In [ ]:
# default_exp linesdk

In [ ]:
#hide
import pickle
accessKey = ''
keypath = '/Users/nic/.lineAccessKey'
if accessKey :
  with open (keypath, 'wb') as f:
    pickle.dump({
        'accessKey': accessKey
    }, f)
with open(keypath, 'rb') as f:
  creden = pickle.load(f)
accessKey = creden['accessKey']
# print(accessKey)

# Products

> interact with product apis

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from lambdasdk.lambdasdk import Lambda
from linebot.models import TextSendMessage
from linebot import LineBotApi
import bz2,  boto3, base64, logging

In [ ]:
#hide
# logging.basicConfig(level=logging.INFO)

In [ ]:
#export
class FunctionNames:
  '''determine function and resources name based on branchName'''
  def __init__(self, branchName:str = 'dev-manual'):
    self.branchName = branchName
  dumpToS3 = lambda self: f'product-dump-s3-{self.branchName}'
  updateProduct = lambda self: f'product-update-{self.branchName}'
  updateS3 = lambda self: f'product-update-s3-{self.branchName}'
  singleQuery = lambda self: f'product-get-{self.branchName}'
  allQuery = lambda self: f'product-get-all-{self.branchName}'
  inputBucket = lambda self: f'input-product-bucket-{self.branchName}'
  inventoryBucket = lambda self: f'product-bucket-{self.branchName}'
  

In [ ]:
#export
class Line:
  '''
    the main class for interacting with product endpoint
    user/pw are optional
  '''
  def __init__(self, 
               accessKey:str = ''):
    self.line_bot_api = LineBotApi(accessKey)
    
  def send(self, message:str = '', roomId:str=''):
    self.line_bot_api.push_message(roomId, TextSendMessage(text = message))
    return True
  @staticmethod
  def lambdaSend(event, _):
    line = Line(accessKey = event['accessKey'])
    line.send(message = event['message'], roomId = event['roomId'])
    


# Testing

## generate dummy data for testing

In [ ]:
#hide
sampleInput = {
  "message": "helloWorld",
  "roomId": 'C9ba1d024ed36979222a2a2a8f67cfc9a' ,
  "accessKey": accessKey
}


## Create main class object

In [ ]:
line = Line(accessKey = accessKey)

## sendMessage

In [ ]:
%%time
result = line.send(roomId = sampleInput['roomId'], message= sampleInput['message'])
print(result)

True
CPU times: user 15.2 ms, sys: 2.38 ms, total: 17.6 ms
Wall time: 339 ms


## Lambda

In [ ]:
%%time
result = Line.lambdaSend(sampleInput, '')

CPU times: user 13.2 ms, sys: 2.09 ms, total: 15.3 ms
Wall time: 798 ms
